<hr style="border:2px solid gray"> </hr>

# Machine Learning
<hr style="border:2px solid gray"> </hr>

# Step 0. Import packages and Global Settings

In [1]:
import os, glob, numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input, Concatenate, AveragePooling2D, DepthwiseConv2D, BatchNormalization, Activation, Add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt


train_name = 'train1.hdf5'
test_name = 'test1.hdf5'
category_file_name = 'Categories.txt'
model_dir = './model'

group = 2

# Step 1. Data Loading

In [2]:
import h5py
from tensorflow.keras.utils import HDF5Matrix

X_train, X_test, y_train, y_test = HDF5Matrix(train_name, 'X'), HDF5Matrix(test_name, 'X'), HDF5Matrix(train_name, 'y'), HDF5Matrix(test_name, 'y')

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

Instructions for updating:
Training with HDF5Matrix is not optimized for performance. Instead, we recommend using https://github.com/tensorflow/io to load your HDF5 data into a tf.data Dataset and passing that dataset to Keras.
(21303, 220, 220, 3) (21303, 296)
(7101, 220, 220, 3) (7101, 296)


In [3]:
import codecs
category_file = codecs.open(category_file_name, 'r', 'utf-8')
categories = list(map(str.strip, category_file.readlines()))
classes = len(categories)

# Step 2. Define Model

In [4]:
l2_ratio = 0.01
with tf.device('/device:GPU:0'):
    model = Sequential([
        Conv2D(64, (5,5), padding="same", input_shape=X_train.shape[1:], activation='relu'),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        #Dropout(0.25),

        Conv2D(128, (3,3), padding="same", activation='relu'),
        Conv2D(128, (3,3), padding="same", activation='relu'),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        #Dropout(0.25),

        Conv2D(256, (3,3), padding="same", activation='relu'),
        Conv2D(256, (3,3), padding="same", activation='relu'),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        #Dropout(0.25),

        Conv2D(256, (3,3), padding="same", activation='relu'),
        Conv2D(256, (3,3), padding="same", activation='relu'),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),

        Flatten(),
        Dense(4096, activation='relu',kernel_regularizer=l2(l2_ratio)),
        Dense(1024, activation='relu',kernel_regularizer=l2(l2_ratio)),
        Dense(classes, activation='softmax',kernel_regularizer=l2(l2_ratio))
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 220, 220, 64)      4864      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 109, 109, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 128)     73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 54, 54, 256)       295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 54, 54, 256)       5

# Training

In [6]:
history = model.fit(X_train[:], y_train[:], batch_size=16, epochs=200, validation_data=(X_test[:], y_test[:]))
print("정확도 : %.4f" % (model.evaluate(test)[1]))

Epoch 1/100
666/666 [==============================] - 224s 337ms/step - loss: 6.9264 - accuracy: 0.0023 - val_loss: 5.6937 - val_accuracy: 0.0021
Epoch 2/100
666/666 [==============================] - 181s 272ms/step - loss: 5.6892 - accuracy: 0.0033 - val_loss: 5.6917 - val_accuracy: 0.0021
Epoch 3/100
666/666 [==============================] - 180s 271ms/step - loss: 5.6873 - accuracy: 0.0039 - val_loss: 5.6923 - val_accuracy: 0.0021
Epoch 4/100
666/666 [==============================] - 182s 273ms/step - loss: 5.6859 - accuracy: 0.0033 - val_loss: 5.6931 - val_accuracy: 0.0021
Epoch 5/100
666/666 [==============================] - 181s 272ms/step - loss: 5.6853 - accuracy: 0.0031 - val_loss: 5.6939 - val_accuracy: 0.0021
Epoch 6/100
360/666 [===============>..............] - ETA: 1:15 - loss: 5.6844 - accuracy: 0.0026

ResourceExhaustedError:  OOM when allocating tensor with shape[36864,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/dense/kernel/Regularizer/Mul (defined at <ipython-input-6-ce86ec615ad3>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_1471]

Function call stack:
train_function


# Test

In [ ]:
def plot_history(histories, key='binary_crossentropy'):
    plt.figure(figsize=(16,10))

    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                    '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
                label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(history.epoch)])

In [ ]:
plot_history([('', history)], 'accuracy')

In [ ]:
model = Sequential([
        Conv2D(32, (7,7), padding="same", input_shape=X_train.shape[1:], activation='relu', strides=(2, 2)),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        #Dropout(0.25),

        Conv2D(64, (4,4), padding="same", activation='relu'),
        Conv2D(64, (4,4), padding="same", activation='relu'),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        #Dropout(0.25),

        Conv2D(128, (2,2), padding="same", activation='relu'),
        Conv2D(128, (2,2), padding="same", activation='relu'),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        #Dropout(0.25),

        Flatten(),
        #Dense(4096, activation='relu',kernel_regularizer=l2(l2_ratio)),
        Dense(1024, activation='relu',kernel_regularizer=l2(l2_ratio)),
        #Dense(512, activation='relu',kernel_regularizer=l2(l2_ratio)),
        Dense(classes, activation='softmax',kernel_regularizer=l2(l2_ratio))
    ])